In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 22.4 MB/s 
     |████████████████████████████████| 182 kB 95.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 


# MultiCoNER II

English to French Translation using T5-small pre-trained model

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer_en_fr = T5Tokenizer.from_pretrained("t5-small")
model_en_fr = T5ForConditionalGeneration.from_pretrained("t5-small")
task_prefix = "translate English to French: "

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [8]:
coner_tags =[
    # Location - LOC
    "B-Facility", "I-Facility", "B-OtherLOC", "I-OtherLOC", "B-HumanSettlement", "I-HumanSettlement", "B-Station", "I-Station",
    # Creative Work - CW
    "B-VisualWork", "I-VisualWork", "B-MusicalWork", "I-MusicalWork", "B-WrittenWork", "I-WrittenWork",
    "B-ArtWork", "I-ArtWork", "B-Software", "I-Software", "B-OtherCW", "I-OtherCW",
    # Group - GRP
    "B-MusicalGRP", "I-MusicalGRP", "B-PublicCORP", "I-PublicCORP", "B-PrivateCORP", "I-PrivateCORP",
    "B-OtherCORP", "I-OtherCORP", "B-AerospaceManufacturer", "I-AerospaceManufacturer", "B-SportsGRP", "I-SportsGRP",
    "B-CarManufacturer", "I-CarManufacturer", "B-TechCORP", "I-TechCORP", "B-ORG", "I-ORG",
    # Person - PER
    "B-Scientist", "I-Scientist", "B-Artist", "I-Artist", "B-Athlete", "I-Athlete", "B-Politician", "I-Politician",
    "B-Cleric", "I-Cleric", "B-SportsManager", "I-SportsManager", "B-OtherPER", "I-OtherPER",
    # Product - PROD
    "B-Clothing", "I-Clothing", "B-Vehicle", "I-Vehicle", "B-Food", "I-Food", "B-Drink", "I-Drink", "B-OtherPROD", "I-OtherPROD",
    # Medical - MED
    "B-Medication/Vaccine", "I-Medication/Vaccine", "B-MedicalProcedure", "I-MedicalProcedure",
    "B-AnatomicalStructure", "I-AnatomicalStructure", "B-Symptom", "I-Symptom", "B-Disease", "I-Disease",
]

In [9]:
in_path = "/content/en-train.conll"

sent_list = []
sentence = ""
with open(in_path, 'r') as inf:
    for line in inf:
      line = line.strip()
      if line != '':
          if line[0] == '#':
              continue
          else:
              line = line.split()
              #print(line)
              if len(line) >= 2:
                  if line[-1] in coner_tags:
                    sentence =  sentence + " " + line[0] + " " + line[-1]
                    #print(sentence)
                  else:
                    sentence = sentence + " " + line[0]
                    #print(sentence)
      else:
        if sentence != "":
          sent_list.append(sentence)
          sentence = ""

print(len(sent_list))

16778


In [10]:
print(sent_list[0])

 robert B-OtherPER gottschalk I-OtherPER 1939 academy B-VisualWork award I-VisualWork winner and founder of panavision B-ORG


In [11]:
# do batch translation
inputs = tokenizer_en_fr([task_prefix + s for s in sent_list ],  max_length=64, truncation=True, return_tensors="pt", padding=True)

output_sequences = model_en_fr.generate(
    max_length=64,
    input_ids=inputs["input_ids"],
    #attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)
translated_list = tokenizer_en_fr.batch_decode(output_sequences, skip_special_tokens=True)

In [12]:
print(len(translated_list))
print(translated_list[2])

16778
d’autres recherches ont conduit dans les années 60 à la représentation bahadur B-OtherPER qui fournit des informations sur les erreurs.


In [13]:
out_path = "/content/en-fr-train.conll"

cnt = 0
with open(in_path, 'r') as inf, open(out_path, 'w') as of:
    for line in inf:
        if line[0] == '#':
            of.write(line)
            of.write(translated_list[cnt] + '\n')
            of.write("\n")
            cnt += 1
        else:
          continue


French to English translation usin Helsinki French to English pre-trained model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_fr_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

model_fr_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en")


/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
in_path_fr = "/content/fr-train.conll"

sent_list_fr = []
sentence = ""
with open(in_path_fr, 'r') as inf:
    for line in inf:
      line = line.strip()
      if line != '':
          if line[0] == '#':
              continue
          else:
              line = line.split()
              #print(line)
              if len(line) >= 2:
                  if line[-1] in coner_tags:
                    sentence =  sentence + " " + line[0] + " " + line[-1]
                    #print(sentence)
                  else:
                    sentence = sentence + " " + line[0]
                    #print(sentence)
      else:
        if sentence != "":
          sent_list_fr.append(sentence)
          sentence = ""

print(len(sent_list_fr))
print(sent_list_fr[1])

16548
 salvien B-Artist de I-Artist marseille I-Artist auteur latin chrétien ( date approximative ) .


In [ ]:
data_len = len(sent_list_fr)
batch_num = data_len // 2000
print(batch_num)

8


In [ ]:
# Since I was running out of RAM, I divided the data into groups of 2000 samples

translated_list_fr_en = []
for i in range(batch_num):
  inputs_fr_en = tokenizer_fr_en([s for s in sent_list_fr[i*2000:i*2000+2000]],  max_length=64, truncation=True, return_tensors="pt", padding=True)

  output_sequences_fr_en = model_fr_en.generate(
      max_length=64,
      input_ids=inputs_fr_en["input_ids"],
      #attention_mask=inputs["attention_mask"],
      do_sample=False,  # disable sampling to test if batching affects output
  )
  temp = tokenizer_fr_en.batch_decode(output_sequences_fr_en, skip_special_tokens=True)
  translated_list_fr_en = translated_list_fr_en + temp

print(len(translated_list_fr_en))

16000


In [ ]:
# The rest of the dataset
inputs_fr_en = tokenizer_fr_en([s for s in sent_list_fr[8*2000:]],  max_length=64, truncation=True, return_tensors="pt", padding=True)

output_sequences_fr_en = model_fr_en.generate(
    max_length=64,
    input_ids=inputs_fr_en["input_ids"],
    #attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)
temp = tokenizer_fr_en.batch_decode(output_sequences_fr_en, skip_special_tokens=True)
translated_list_fr_en = translated_list_fr_en + temp
print(len(translated_list_fr_en))

16548


In [ ]:
print(len(translated_list_fr_en))
print(translated_list_fr_en[2])

16548
it is presented alternately by annette B-Artist Gerlach I-Artist gustav B-Artist hofer I-Artist and marie B-OtherPER laboratory I-OtherPER.


In [ ]:
out_path_fr_en = "/content/fr-en-train.conll"

cnt = 0
with open(in_path_fr, 'r') as inf, open(out_path_fr_en, 'w') as of:
    for line in inf:
        if line[0] == '#':
            of.write(line)
            of.write(translated_list_fr_en[cnt] + '\n')
            of.write("\n")
            cnt += 1
        else:
          continue


**Use tools/line2cols.py to change the format of the samples. This code will return sentences (one line per sample), line2col.py will change a line to columns , entities along with their tags**